# First-Level Analysis

This lesson is one of two lessons on how to run conventional mass-univariate analyses using general linear models (called GLM analysis). Let's break this down. Mass-univariate refers to performing univariate analysis on each voxel. The way this is often done is by fitting a GLM to each voxel (recall [link](http://129.100.119.110:22028/GLM-LocalizerPred)), although there are certainly other statistical approaches you can do. In doing so, mass-univariate analysis can identify voxels in the brain that are 'modulated' or differentially 'activated' by a task. This is sometimes called 'blobology' because it generates 'blobs', or clusters, in the brain of significant voxels. 

Pretty much all fMRI analyses involve some sort of analysis on the individual subjects (first-level analysis), which is followed by some sort of group-analysis (second-level analysis). The latter seeks to identify group-level effects of whatever was done in the former. 

This lesson focuses on first-level GLM analysis -- that is, taking a subject's brain and fitting GLMs for each of their voxels. We will start by doing a step-by-step walkthrough on one subject. The key here is to really take the time to understand both the concepts and the implementation. When I started, I personally found GLM analyses pretty confusing, so don't worry if you get lost! We will cover each step in detail, one chunk of code at a time.  

Afterwards, we'll run first-level analyses on all of the subjects. Subjects' data will be saved and used as input in the next lesson, where we will perform our second-level analysis. The implementation we will use is pretty much what you would do in an actual project. 

---
**A note on other fMRI software**:

Typically, people use software such as AFNI, SPM or FSL for mass-univariate analyses. While these packages serve their purpose well, they come with their own set of hurdles:
- Each software has their own learning curves
- Documentation can be sparse, non-existent, or hard to digest 
- Practical tutorials and examples can be hard to come by  
- Their GUI implementations (FSL and SPM) tend to obfuscate analysis and can be very tedious
- SPM requires a MATLAB license, and AFNI and FSL cannot be run directly on Windows

As noted in previous lessons, I recommend trying out one of these software packages in your own time just to get a sense of what's out there. They're full fMRI packages that each come with some excellent tools, which can come in handy. 

---

Here, we will be implementing mass-univariate analyses using nilearn. Running mass-univariate analyses is straightforward with nilearn, and importantly, it doesn't hide a lot of details. By being so explicit, it easy to check your steps. I think it does a great job at keep your analyses transparent. 

Let's start by importing all of our functions. You'll notice that there are many more than previous lessons. We're importing:
- numpy and nibabel (as usual)
- glob ([link](https://docs.python.org/3/library/glob.html)), which will allow us to pattern match for files
- matplotlib just to help with some visualization
- pandas, which we'll use to read and adjust some files
- nilearn's `glm` functions/classes to set up and run the analysis 
- pickle ([link](https://docs.python.org/3/library/pickle.html)), which lets us save Python objects to disk

In [ ]:
import glob
import pickle
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nilearn.plotting import view_img, plot_design_matrix
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix

## 1. In-depth single subject walkthrough

We'll go through each step of performing a first-level analysis of the motor localizer in one subject.  

### 1.1. Setting up the data

We'll first get the preprocessed functional runs of one task produced by fmriprep. We can use Python's glob module to extract all files matching a template. Basically, it will find all files that match the template in the string, where `*` is a wildcard that says any number of any character can be filled in there. The template we'll provide will give us the preprocessed motor localizer runs.  

In [ ]:
glob_template = ('course-data/derivatives/fmriprep/sub-01/func/'
                '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz') #specify the template of the files we are looking for
func_data = glob.glob(glob_template) #grab functional data filesnames

# reorder these based on run number 
func_imgs = sorted(list(func_data))
func_imgs #note that these functional images point to the MNI-transformed BOLD data that has been preprocessed


Next, we will also need the run's motion parameters that were extracted during motion correction in preprocessing. fmriprep stores all of the parameters, along with many others, in the `confounds_regressors.tsv` files. Each run has one of these files. 

We will use Pandas to load in the file into Python and select only the motion parameters. 

In [ ]:
glob_template = ('course-data/derivatives/fmriprep/sub-01/func/'
                '*task-motor*confounds_regressors.tsv')
conf_files = glob.glob(glob_template) #grab confound filenames

# reorder these based on run number
conf_files = sorted(list(conf_files))

# load only the 6 motion parameters into python
motion_params = []
for conf in conf_files:
    conf_data = pd.read_table(conf, usecols=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'])
    motion_params.append(conf_data) #append the dataframe to motion_params

# show the first one
motion_params[0]


In [ ]:
# let's plot the motion parameters for this one run:

# Create a figure with two subplots (one for translations, one for rotations)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot translations
motion_params[0][['trans_x', 'trans_y', 'trans_z']].plot(ax=ax1)
ax1.set_title('Translations')
ax1.set_ylabel('mm')
ax1.legend(['x', 'y', 'z'])

# Plot rotations
motion_params[0][['rot_x', 'rot_y', 'rot_z']].plot(ax=ax2)
ax2.set_title('Rotations')
ax2.set_ylabel('radians')
ax2.legend(['x', 'y', 'z'])

plt.tight_layout()
plt.show()

### 1.2. Defining the model

Before we set up a model, let's load in a brain mask of the MNI template. This mask will restrict our analysis to voxels only inside the brain.

In [ ]:
brain_mask = nib.load('MNI_brain_mask.nii.gz')
#print(brain_mask)

Next, we will specify our model. Nilearn makes it really easy to specify a GLM. The `FirstLevelModel` class let's you define all your parameters in one step, and then you can use it's methods to fit the model and define contrasts. 

The [online documentation](https://nilearn.github.io/stable/modules/generated/nilearn.glm.first_level.FirstLevelModel.html#nilearn.glm.first_level.FirstLevelModel) for the `FirstLevelModel` gives a description for each parameter. For our analysis, a few things are required:
- The TR, which is 2s
- The HRF model, in which there are several to choose from (see documentation for the options). We will choose Glover's (1999) canonical HRF along with the temporal derivative. Adding the derivative is often recommended to improve model fit. The option we'll use is 'glover + derivative'. 
- We also want to apply some additional processing steps, on top of what was done with fmriprep. We will scale and temporally filter the data (0.01Hz), and spatially smooth the data using a 6mm FWHM kernel. 
- The brain mask we just loaded

Some of the above options are the default parameters, but we'll explicitly specify them. 

When you run this, nothing will happen! Running the below command simply configures the analysis.

In [ ]:
model = FirstLevelModel(t_r=2, hrf_model='glover + derivative', high_pass=.01, 
                        signal_scaling=0, smoothing_fwhm=6, mask_img=brain_mask, verbose=1)
# signal_Scaling=0 is the default. It means that the fMRI time series for each voxel will be scaled so that its mean is 0, and its values are expressed 
# as percent signal change relative to that mean.

### 1.3 The model design

The design matrix is what defines our regressors. We are trying to _predict actual brain activity_ using an a model of activity that we would expect to arise from the task. Each condition (AKA trial type) is defined as a single regressor. In our case, we will have 8 task regressors of interest (Arm, Eye, Finger, Grasp, Mouth, Speech, Toes, Touch). The remaining regressors are there to either improve model fit or regress out noise (i.e. act as temporal filters).

In nilearn, there are a few different ways to build a design matrix. You can build one directly using the `make_first_level_design_matrix` function (see [documentation](https://nilearn.github.io/stable/modules/generated/nilearn.glm.first_level.make_first_level_design_matrix.html#nilearn.glm.first_level.make_first_level_design_matrix)). I'd recommend trying this in your own time because it's nice to build your design matrix and inspect it before running your analysis.

For this lesson, we'll use the much simpler way of specifying a design matrix: we will by pass in events files when fitting the model. The downside to this approach is that we can only inspect our design matrix _after_ the model has finished fitting.

Let's load in the events files that tell us each trial/block's onset, duration, and condition. There is one events file for each run, which we can load in a for-loop using pandas' `read_table` function.

In [ ]:
event_files = glob.glob('course-data/task-motor*events.tsv')
event_files = sorted(list(event_files))

events = []
for ev in event_files:
    events.append(pd.read_table(ev))

print(event_files)

In [ ]:
# show the first 10 rows
events[0].head(10)

We see that this is a block design (each block 20s each) with  and that there are 32 blocks (4 each condition). When we pass this DataFrame into our model, it will be transformed to a proper design matrix.

### 1.4 Fitting the model

We fit the model (or specifically, the model for each voxel) by calling the `.fit()` method. In this method, we pass in the functional image, our events data, and the motion parameters that we saved earlier. Note that this will produce a bunch of warnings--ignore them.

In [ ]:
model.fit(func_imgs, events=events, confounds=motion_params)

Now that we've run our model, we can inspect the design matrix. _It's really important to check your design matrix to ensure that it is correct_. Small errors, which can easily be spotted by looking at the design matrix in the design, can have large effects on your results.

In [ ]:
design = model.design_matrices_[0]
fig, ax = plt.subplots(figsize=(13, 8))
plot_design_matrix(design, ax=ax)
plt.show()


The design matrix accurately reflects what we saw in the events file. 

In [ ]:
# Next, let's look at the HRFs for one of the experimental conditions -- ARM (and its derivative)

fig, ax = plt.subplots(figsize=(10, 3))  # Create a figure and an axes object
ax.plot(design['Arm'], label='Arm')  # Plot 'Arm', add a label
ax.plot(design['Arm_derivative'], label='Arm Derivative')  # Plot 'Arm_derivative', add a label

# Set labels and title
ax.set(title='Arm and Arm Derivative', xlabel='Time (s)', ylabel='Signal (a.u.)')
ax.legend()  # Add a legend to show which line is which
plt.show()

# Let's do the same for the Toes condition
fig, ax = plt.subplots(figsize=(10, 3))  # Create a figure and an axes object
ax.plot(design['Toes'], label='Toes')  # Plot 'Arm', add a label
ax.plot(design['Toes_derivative'], label='Toes Derivative')  # Plot 'Arm_derivative', add a label

# Set labels and title
ax.set(title='Toes and Toes Derivative', xlabel='Time (s)', ylabel='Signal (a.u.)')
ax.legend()  # Add a legend to show which line is which
plt.show()


You will have noticed in the above design matrix that there were a bunch of 'drift' regressors. Those relate to the HPF that we specified in our model (i.e., 0.01 Hz). Let's plot these below.

In [ ]:
# plot the drift regressors (included in the design matrix to model and remove unwanted sources of variance in the fMRI data, 
# allowing for a more accurate estimation of the effects of your experimental conditions
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(design['drift_1'], label='drift_1')
ax.plot(design['drift_2'], label='drift_2')
ax.plot(design['drift_3'], label='drift_3')
ax.plot(design['drift_4'], label='drift_4')

# Set labels and title
ax.set(title='HPF regressors', xlabel='Time (s)', ylabel='nuisance regressors (a.u.)')
ax.legend()  # Add a legend to show which line is which
plt.show()

fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(design['drift_10'], label='drift_11')
ax.plot(design['drift_11'], label='drift_12')
ax.plot(design['drift_12'], label='drift_13')
ax.plot(design['drift_14'], label='drift_14')

# Set labels and title
ax.set(title='HPF regressors', xlabel='Time (s)', ylabel='nuisance regressors (a.u.)')
ax.legend()  # Add a legend to show which line is which
plt.show()

You might be asking yourself why are these created. When running a GLM, we actually implement the HPF within the GLM itself (by adding drift regressors) rather than by directly filtering the fMRI data before fitting the GLM data. Why?

**Flexibilty and control within the GLM framework**
1. **Simultaneous Estimation**: The most important reason is that including the drift regressors in the GLM allows for simultaneous estimation of both the drift components and the task-related effects (your regressors of interest, like 'Arm'). This is crucial because it accounts for the potential correlation between the drift and your task regressors.
2. **Avoiding Bias**: If you were to high-pass filter the data separately before the GLM, you could inadvertently remove some of the true task-related signal, especially if your task has low-frequency components or if there's any correlation between the task timing and the low-frequency drifts. The GLM approach avoids this bias by modeling both simultaneously.
3. **Optimal Weighting**: The GLM finds the optimal weights (beta values) for all regressors (both task-related and drift) in a single, unified estimation procedure. This ensures that the drift regressors remove the appropriate amount of low-frequency variance without unduly influencing the estimates of the task-related effects. This is also very computationally efficient.

**Statistical considerations**
1. **Degrees of Freedom**: When you high-pass filter the data directly, you effectively "use up" degrees of freedom in the data. The GLM approach correctly accounts for the degrees of freedom used by the drift regressors during statistical inference (e.g., when calculating t-statistics and p-values). This ensures that your statistical tests are properly calibrated.
2. **Error Structure**: The GLM makes assumptions about the error structure of the data (typically assuming that the errors are independent and identically distributed, or i.i.d., after accounting for autocorrelation). Filtering the data outside the GLM can violate these assumptions and lead to incorrect statistical inferences.


### 1.5 Inspecting regression coefficients

Next we can look at the regression coefficients (remember this was the whole point of running this voxel-by-voxel GLM). If we look at the 'Arm' condition, we can see which voxels match closely to our 'Arm' regressor. That is, voxels with large positive values are likely modulated by an arm movement.

In [ ]:
beta_map = model.compute_contrast('Arm', output_type='effect_size')
view_img(beta_map, resampling_interpolation='nearest', vmax=3)

In [ ]:
# Save the beta map as a NIfTI file
nib.save(beta_map, 'beta_map_Arm.nii.gz')

This code above uses `compute_contrast` to estimate the magnitude of the BOLD response for the 'Arm' condition. By specifying only the 'Arm' regressor, it implicitly performs a one-sample t-test, comparing the 'Arm' condition's activity against a baseline of zero, after accounting for other regressors in the model.  The output, `beta_map`, is an image where each voxel's value represents the effect size (beta value) for the 'Arm' condition.

However, if we could also think of comparing the 'Arm' condition to another condition in our design (e.g., 'Toes'). Let's try that.

In [ ]:
beta_map = model.compute_contrast('Arm - Toes', output_type='effect_size')
view_img(beta_map, resampling_interpolation='nearest', vmax=3)

In [ ]:
# Save the beta map as a NIfTI file
nib.save(beta_map, 'beta_map_ArmVsToes.nii.gz')

We can now see that this beta_map looks a bit different.

*Thought question*: What do the blue (negative effect size) voxels represent in this map?

## 2. Putting it all together 

We'll copy and paste what we did earlier all into one notebook cell. Everything will be in one for-loop that iterates through each subject. Exactly like before, we'll do the following for each subject:
1. Get the subject's functional images and load them
2. Get the subject's 6 motion parameters
3. Set up a model identical to the one we were using before

The events files are the same across subjects, so we don't need to load those in again. We can just reuse them for each subject. As such, we don't need to plot the design matrix for each subject either.

In the glob templates for steps 1) and 2), you'll notice that the string starts with an `f` and contains curly braces: `{subject}`. Python will place the `subject` variable of the iteration into the string (e.g., `'01` in the first iteration of the for-loop). 

Note: If you're unfamiliar with for-loops or large chunks of code, don't be alarmed. Take the time to go through each line and identify it's purpose. All we're doing here is applying the same things as earlier, just one at a time for all subjects. 

In [ ]:
subject_numbers = ['01', '02', '03', '04', '05', '06', '07', '08']
columns = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']

glms = []
for subject in subject_numbers: #looping over each subject's directory
    
    ## 1. Get the functional images in order
    glob_template = (f'course-data/derivatives/fmriprep/sub-{subject}/func/'
                     '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz')
    func_imgs = glob.glob(glob_template)
    func_imgs = sorted(list(func_imgs))
    
    # 2. Get the regressors
    glob_template = (f'course-data/derivatives/fmriprep/sub-{subject}/func/'
                      '*task-motor*confounds_regressors.tsv')
    conf_files = glob.glob(glob_template)
    # put files in order
    conf_files = sorted(list(conf_files))

    # load only the 6 motion parameters
    motion_params = []
    for conf in conf_files:
        conf_data = pd.read_table(conf, usecols=columns)
        motion_params.append(conf_data)
    
    
    # 3. run model with the same parameters
    model = FirstLevelModel(t_r=2, hrf_model='glover + derivative', high_pass=.01, 
                           signal_scaling=0, smoothing_fwhm=6, mask_img=brain_mask)
    # events are the same as we used in our waltkthrough
    model.fit(func_imgs, events=events, confounds=motion_params)
    glms.append(model)
    

### 2.1 Saving the models 

We'll save these models to a file to use them for next week when perform a second-level analysis and discuss multiple comparisons. We can save Python objects to disk using the [pickle package](https://docs.python.org/3/library/pickle.html) that comes as part of the Python standard library. A pickle file (`.p`) is Python's equivalent to MATLAB's `.mat` or R's `.Rdata`. Much like how we can pickle a vegetable for long-term storage, we can pickle Python objects :). 

The file we will make will be ~600 MB because it contains everything belonging to our first-level models. Make sure you have the space available. In a typical project it makes sense to combine first and second level analysis into the same script so that you're not saving off unnecessary files.

In [ ]:
with open('first_level_models.p', 'wb') as f:
    pickle.dump(glms, f)

## 3. Excercise: Tinkering around with paramters

#### Exercise 1: Vary the High-Pass Filter Cutoff
Explore how different high-pass filter (HPF) cutoffs affect your GLM analysis. As you will see, changing its cutoff can influence the design matrix and ultimately the beta maps.

**Steps:**
1. Choose three different HPF values (e.g., 0.005, 0.01, and 0.2 Hz).
2. For each HPF value:
   - Set up and fit the `FirstLevelModel` for one subject.
   - Visualize the drift regressors in the design matrix.
   - Compute and display the beta map for the 'Arm - Toes' contrast.
3. Compare the results

#### Exercise 2: Experiment with Spatial Smoothing Levels
Assess the effect of different spatial smoothing kernels on your GLM results. As we've discussed, spatial smoothing can affect both the spatial extent and detectability of activation clusters.

**Steps:**
1. Choose a subject and fix the high-pass filter (e.g., 0.01 Hz) and HRF model.
2. Change the `smoothing_fwhm` parameter to different values (e.g., 4, 8, and 12 mm).
3. For each smoothing level:
   - Fit the model.
   - Compute and display the beta map for the 'Arm - Toes' contrast.
4. Compare the beta maps

#### Exercise 3: Test Alternative HRF Models

Explore how the choice of the Hemodynamic Response Function (HRF) model affects the GLM analysis. 

**Steps:**
1. Choose a subject and set fixed parameters for high-pass filtering (e.g., 0.01 Hz) and smoothing (4 mm).
2. Fit the GLM using at least two different HRF models (e.g., `'glover'` vs. `'glover + derivative'`).
3. For each HRF model:
   - Fit the model.
   - Visualize the design matrix (focus on the task regressors).
   - Compute and display the beta map for the 'Arm - Toes' contrast.
4. Compare the beta maps

#### Exercise 4: Modify the Set of Confound Regressors

Investigate the impact of including additional confound regressors in your GLM. While the six motion parameters are commonly used, try the effect of including signals from cerebrospinal fluid (CSF) and White Matter (WM).

**Steps:**
1. Identify the specified additional confounds in your TSV files. 
2. Update your code to load these additional columns along with the 6 motion parameters.
3. Fit the GLM model using the expanded set of confounds.
4. Compare the your previous beta maps for the 'Arm - Toes' contrast with this newest one with the expanded set of motion parameters.